In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import Input
from tensorflow.keras import backend as K

In [2]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Generator
def build_generator(input_dim, output_dim):
    model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_dim=input_dim),
        layers.Dense(512, activation='relu'),
        layers.Dense(output_dim, activation='tanh')  # Normalized range
    ])
    return model

# Discriminator
def build_discriminator(input_dim):
    model = tf.keras.Sequential([
        layers.Dense(512, activation='relu', input_dim=input_dim),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Real or fake
    ])
    return model

# GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = layers.Input(shape=(generator.input_shape[1],))
    x = generator(gan_input)
    gan_output = discriminator(x)
    return tf.keras.Model(gan_input, gan_output)

# Training function
def train_gan(generator, discriminator, gan, real_data, epochs=10000, batch_size=32):
    for epoch in range(epochs):
        # Sample real data
        idx = np.random.randint(0, real_data.shape[0], batch_size)
        real_samples = real_data[idx]

        # Generate fake data
        noise = np.random.normal(0, 1, (batch_size, generator.input_shape[1]))
        fake_samples = generator.predict(noise)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((batch_size, 1)))
        d_loss = 0.5 * (d_loss_real + d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, generator.input_shape[1]))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print losses periodically
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")

# Example Usage
input_dim = 100  # Random noise dimension
output_dim = real_data.shape[1]  # Feature size

generator = build_generator(input_dim, output_dim)
discriminator = build_discriminator(output_dim)
gan = build_gan(generator, discriminator)

discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Train GAN
train_gan(generator, discriminator, gan, real_data, epochs=10000, batch_size=32)

# Generate new samples
noise = np.random.normal(0, 1, (10, input_dim))
new_samples = generator.predict(noise)
